In [241]:
import numpy as np
import pandas as pd
import os
from gensim.test.utils import get_tmpfile
from gensim.models import Word2Vec
import itertools
from sklearn.cluster import KMeans
import matplotlib
import matplotlib.pyplot as plt

n = 2
w = 6

In [242]:
path = r"C:\Users\oisin\Code\CS185CFinal\data"

In [239]:
z = 5

In [3]:
abspath, folders, files = next(os.walk(path))
topFolders = [os.path.join(abspath, f) for f in folders]
topFiles = [f for f in files if not f.startswith('.')]

In [4]:
exec(f'top20 = {open(os.path.join(abspath, topFiles[0])).read()}')

In [5]:
top20 = [x.lower() for x in top20]

In [6]:
data = dict(
    (folders[f], os.listdir(g)) for f, g in enumerate(topFolders))

In [7]:
def parse_files(folder, file):
    return np.array(open(os.path.join(path, folder, file)).read().split())

class DataIterator():
    
    def __init__(self, malwareType):
        self.malwareType = malwareType
        self.filelist = data[malwareType]
        self.listiter = iter(self.filelist)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        return parse_files(self.malwareType, next(self.listiter))

In [165]:
combos = list(itertools.combinations(top20, 2))

In [182]:
def modelGen(malwareType):
    # Data Iterator for Word2Vec
    d = DataIterator(malwareType)
    
    # Word2Vec
    fname = f'{malwareType}_w2v.model'
    file = get_tmpfile(fname)
    model = Word2Vec(d, size=n, window=w, workers=8)
    model.save(fname)
    
    # DataFrame of cosine similarities
    model_sim = pd.DataFrame(columns=top20, index=top20)
    for i, j in combos:
        model_sim[i][j] = model_sim[j][i] = model.wv.n_similarity([i], [j])
    model_sim = model_sim.fillna(1.0)
    
#     # Run k-means to cluster values
#     kmeans = KMeans().fit(np.array(model_sim['Score']).reshape(-1, 1))
#     model_sim['Label'] = kmeans.labels_
    
#     # Plot for funsies
#     X, Y = model_sim['Score'], np.zeros_like(model_sim['Score'])
#     plt.scatter(X, Y, c=model_sim['Label'], cmap="Dark2")
#     plt.yticks([])
    
#     # Group the codes by their k-means label
#     grouping = model_sim.groupby('Label')
#     grouping = [list(grouping.get_group(key)['Code']) for key, item in grouping]
    return {"file": file, "model": model, "cos_sim": model_sim}#, "plot": plt, "groups": grouping}

In [183]:
model_gen = dict((f, modelGen(f)) for f in folders)

In [223]:
def color(x):
    if x == '-':
        return ""
    if x >= 0.95:
        return "background: red"
    if 0.75 <= x and x < 0.95:
        return "background: orange"
    if 0.50 <= x and x < 0.75:
        return "background: yellow"
    return ""

test_df = model_gen['CeeInject']['cos_sim']

In [224]:
test_df.style.apply(lambda x: [color(v) for v in x], axis = 1)

,push,mov,inc,dec,pop,add,xor,xchg,or,adc,sub,and,sbb,cmp,imul,out,outsl,jo,js,insl
push,nan,-0.361421,0.897999,-0.487267,-0.965182,-0.396261,-0.971041,0.79189,0.357626,-0.975608,-0.984859,-0.930399,-0.284889,0.562097,0.59401,0.204353,-0.930755,0.987754,-0.178734,-0.603922
mov,-0.361421,nan,-0.734811,-0.638115,0.592734,0.999291,0.128191,-0.85559,0.741484,0.557288,0.517588,-0.00550461,0.996729,-0.974317,-0.964766,-0.986584,-0.00453192,-0.211524,0.981987,-0.524895
inc,0.897999,-0.734811,nan,-0.0533352,-0.981826,-0.759821,-0.766872,0.979807,-0.0897508,-0.972683,-0.960679,-0.674216,-0.677595,0.868672,0.88738,0.614222,-0.674934,0.818354,-0.593416,-0.191623
dec,-0.487267,-0.638115,-0.0533352,nan,0.241877,-0.608681,0.681788,0.147402,-0.98976,0.283684,0.328505,0.773442,-0.698249,0.448351,0.413054,0.75525,0.772825,-0.617543,-0.7721,0.990292
pop,-0.965182,0.592734,-0.981826,0.241877,nan,0.62263,0.874736,-0.924055,-0.100896,0.999061,0.995915,0.802123,0.525709,-0.75887,-0.783756,-0.453297,0.802704,-0.912552,0.429878,0.374405
add,-0.396261,0.999291,-0.759821,-0.608681,0.62263,nan,0.16543,-0.874469,0.715703,0.588146,0.549427,0.0321391,0.992981,-0.982103,-0.973986,-0.97974,0.0331114,-0.248163,0.974179,-0.492485
xor,-0.971041,0.128191,-0.766872,0.681788,0.874736,0.16543,nan,-0.623062,-0.570383,0.894908,0.914921,0.991029,0.0476256,-0.34822,-0.384611,0.0354364,0.991158,-0.996424,-0.0615086,0.776857
xchg,0.79189,-0.85559,0.979807,0.147402,-0.924055,-0.874469,-0.623062,nan,-0.287076,-0.906628,-0.885763,-0.512936,-0.810959,0.950181,0.961643,0.759602,-0.513771,0.686918,-0.742368,0.00848574
or,0.357626,0.741484,-0.0897508,-0.98976,-0.100896,0.715703,-0.570383,-0.287076,nan,-0.1439,-0.19032,-0.675042,0.793282,-0.571352,-0.538821,-0.841075,-0.674324,0.498947,0.854907,-0.960309
adc,-0.975608,0.557288,-0.972683,0.283684,0.999061,0.588146,0.894908,-0.906628,-0.1439,nan,0.998892,0.827239,0.488364,-0.729946,-0.756116,-0.414258,0.827785,-0.929411,0.390361,0.414222


In [227]:
tdf = test_df.fillna(0.0)
tdf.stack().index[np.argmax(tdf.values)]

('and', 'outsl')

,push,mov,inc,dec,pop,add,xor,xchg,or,adc,sub,and,sbb,cmp,imul,out,outsl,jo,js,insl
jo,0.987754,-0.211524,0.818354,-0.617543,-0.912552,-0.248163,-0.996424,0.686918,0.498947,-0.929411,-0.945752,-0.976194,-0.131847,0.426175,0.461225,0.049125,-0.976404,0.000000,-0.023039,-0.720879
inc,0.897999,-0.734811,0.000000,-0.053335,-0.981826,-0.759821,-0.766872,0.979807,-0.089751,-0.972683,-0.960679,-0.674216,-0.677595,0.868672,0.887380,0.614222,-0.674934,0.818354,-0.593416,-0.191623
xchg,0.791890,-0.855590,0.979807,0.147402,-0.924055,-0.874469,-0.623062,0.000000,-0.287076,-0.906628,-0.885763,-0.512936,-0.810959,0.950181,0.961643,0.759602,-0.513771,0.686918,-0.742368,0.008486
